In [1]:
# Imports
import requests
import pandas
import os
import datetime
import numpy as np

In [2]:
# GLOBAL VARS: 

NSE_URL1 = 'https://www1.nseindia.com/products/content/equities/equities/eq_security.htm'

HEADER_REQ1 = {
"Host" : "www1.nseindia.com",
"Connection" : "keep-alive",
"sec-ch-ua":   "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Google Chrome\";v=\"90\"",
"sec-ch-ua-mobile": "?0",
"Upgrade-Insecure-Requests" : "1",
"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
"Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site" : "none",
"Sec-Fetch-Mode" : "navigate",
"Sec-Fetch-User" : "?1",
"Sec-Fetch-Dest" : "document",
"Accept-Encoding" : "gzip, deflate, br",
"Accept-Language" : "en-US,en;q=0.9"
}

DEBUG = True 
INCLUDE_NORMAL_ACTIVITY = True

In [3]:
def iqr_Anomaly_Upper(datalist,sample):
    if (datalist is None):
        return -1
    # convert to Numpy array. Easy to apply qunatile funtion 
    datalist = np.array(datalist)
    
    Q3, Q1 = np.percentile(datalist, [75 ,25])
    IQR = Q3 - Q1
    
    IQR_ANOMALY_UPPER_THRESHOLD = Q3 + 1.5 * IQR

    if(sample>=IQR_ANOMALY_UPPER_THRESHOLD):
        return 1  # Sample shows high volume activity 
    else:
        return 0  # Sample is normal volume activity 

# Format of the result row: 
# Stockname, Trading Verdit, Delivery Verdict
# ICICIBANK,ABNOMALLY HIGH TRADING,ABNOMALLY HIGH DELIVERY
# DMART,NORMAL TRADING,NOMAL DELIVERY
# DMART,HIGH TRADING,HIGH DELIVERY
# Thresholds = {NORMAL, HIGH, ABNOMALLY HIGH}
def iqrAnomalyCheck(stockname,date,df):

    if not isinstance(df, pandas.DataFrame):
        return str(stockname+","+date+",INVALID DATA,INVALID DATA,INVALID DATA\n")
    if (df is None):
        return str(stockname+","+date+",NULL DATA,NULL DATA,NULL DATA\n")
    if (len(df.index)<7): 
        return str(stockname+","+date+",TOO LESS DATA,TOO LESS DATA,TOO LESS DATA\n")
    #Algorithm for checking {NORMAL, HIGH, ABNOMALLY HIGH}
    
    Trade_Volume_Latest_Sample          = df["Total Traded Quantity"].to_numpy()[-1]
    Trade_Delivery_Latest_Sample        = df["DeliverableQty"].to_numpy()[-1]
    Pct_Delivery_To_Trade_Latest_Sample = df["% Dly Qt toTraded Qty"].to_numpy()[-1]
    
    Trade_Volume   = df["Total Traded Quantity"].to_numpy()[:-1]
    Trade_Delivery = df["DeliverableQty"].to_numpy()[:-1]
    Trade_Pct      = df["% Dly Qt toTraded Qty"].to_numpy()[:-1]
    
    
    Q3_TV, Q1_TV = np.percentile(Trade_Volume, [75 ,25])
    TV_IQR_ANOMALY_UPPER_THRESHOLD = Q3_TV + 1.5 * (Q3_TV - Q1_TV) # UPPER HARD THRESHOLD for Trade Volume
    TV_N95_UPPER_THRESHOLD =  np.percentile(Trade_Volume, 95) # UPPER SOFT THRESHOLD for Trade Volume
    
    
    Q3_TD, Q1_TD = np.percentile(Trade_Delivery, [75 ,25])
    TD_IQR_ANOMALY_UPPER_THRESHOLD = Q3_TD + 1.5 * (Q3_TD - Q1_TD) # UPPER HARD THRESHOLD for Trade Volume
    TD_N95_UPPER_THRESHOLD =  np.percentile(Trade_Delivery, 95) # UPPER SOFT THRESHOLD for Trade Volume
    
    #Q3_PCT, Q1_PCT = np.percentile(Trade_Pct, [75 ,25])
    #PCT_IQR_ANOMALY_UPPER_THRESHOLD = Q3_PCT + 1.5 * (Q3_PCT - Q1_PCT) # UPPER HARD THRESHOLD for Delivery PCT
    #PCT_N95_UPPER_THRESHOLD =  np.percentile(Trade_Pct, 95) # UPPER SOFT THRESHOLD for Delivery PCT
    
    
    # Check Thresholds for Trading: 
    Trade_Volume_Outcome = "NOMAL TRADING"
    Trade_Delivery_Outcome = "NOMAL DELIVERY"
    Trade_PCT_Outcome = "NORMAL DELIVERY PERCENTAGE"
    
    if (Trade_Volume_Latest_Sample>=TV_IQR_ANOMALY_UPPER_THRESHOLD): 
        Trade_Volume_Outcome = "ABNOMALLY HIGH TRADING"
    elif (Trade_Volume_Latest_Sample>=TD_N95_UPPER_THRESHOLD):
        Trade_Volume_Outcome = "HIGH TRADING"
    else:
        Trade_Volume_Outcome = "NORMAL TRADING"
        
    # Check Thresholds for Delivery: 
    if (Trade_Delivery_Latest_Sample>=TD_IQR_ANOMALY_UPPER_THRESHOLD): 
        Trade_Delivery_Outcome = "ABNOMALLY HIGH DELIVERY"
    elif (Trade_Delivery_Latest_Sample>=TD_N95_UPPER_THRESHOLD):
        Trade_Delivery_Outcome = "HIGH DELIVERY"
    else:
        Trade_Delivery_Outcome = "NORMAL DELIVERY"
        
    # Check Thresholds for Percentage Delivery: 
    if (Pct_Delivery_To_Trade_Latest_Sample>=90):   # If >90% then good
        Trade_PCT_Outcome = "ABNOMALLY HIGH DELIVERY PERCENTAGE"
    elif (Pct_Delivery_To_Trade_Latest_Sample>=70):
        Trade_PCT_Outcome = "HIGH DELIVERY PERCENTAGE"
    else:
        Trade_PCT_Outcome = "NORMAL DELIVERY PERCENTAGE"    

    return str(stockname+","+date+","+Trade_Volume_Outcome+","+Trade_Delivery_Outcome+","+Trade_PCT_Outcome+"\n")

In [4]:
def fetchDataFromNSE(_symbol_list,_lastndays=21): 
    
    
    date_past = str(datetime.date.today() - pandas.offsets.DateOffset(days=_lastndays)).split(" ")[0]
    date_today  = str(datetime.date.today() - pandas.offsets.DateOffset(days=0)).split(" ")[0]
    
    date_past_obj = datetime.datetime.strptime(date_past, '%Y-%m-%d')
    date_today_obj = datetime.datetime.strptime(date_today, '%Y-%m-%d')
    
    
    date_past = date_past_obj.strftime('%d-%m-%Y')
    date_today = date_today_obj.strftime('%d-%m-%Y')

    print("date from ", date_past, type(date_past))
    print("date today ",date_today, type(date_today))
    
    
    #_symbol = "DMART"
    _segmentLink = "3"
    _symbolCount = "1"
    _series = "EQ"
    _dateRange = "+"
    _fromDate = date_past
    _toDate = date_today
    _dataType = "PRICEVOLUMEDELIVERABLE"
    
 
    
    # For debugging
    #print(f"===Start of Params===")
    ##print(f"got symbol:{_symbol}")
    #print(f"got segmentLink:{_segmentLink}")
    #print(f"got symbolCount:{_symbolCount}")
    #print(f"got series:{_series}")
    #print(f"got dateRange:{_dateRange}")
    #print(f"got fromDate:{_fromDate}")
    #print(f"got toDate:{_toDate}")
    #print(f"got dataType:{_dataType}")
    #print(f"===End of Params===")
    
       
 
    sess = requests.Session()
    rs = sess.get(NSE_URL1, headers=HEADER_REQ1)
    
    arr_cookies = [{'name': c.name, 'value': c.value, 'domain': c.domain, 'path': c.path, 'expires': c.expires} for c in sess.cookies]
    parsed_cookies = arr_cookies[0].get('name') + "=" + arr_cookies[0].get('value')
    
    custom_headers = {
            "Host" : "www1.nseindia.com",
            "Connection" : "keep-alive",
            "sec-ch-ua":   "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Google Chrome\";v=\"90\"",
            "sec-ch-ua-mobile": "?0",
            "Upgrade-Insecure-Requests" : "1",
            "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
            "Accept" : "*/*",
            "X-Requested-With" : "XMLHttpRequest",
            "Sec-Fetch-Site" : "same-origin",
            "Sec-Fetch-Mode" : "cors",
            "Sec-Fetch-Dest" : "empty" ,
            "Referer" : "https://www1.nseindia.com/products/content/equities/equities/eq_security.htm",
            "Accept-Encoding" : "gzip, deflate, br",
            "Accept-Language" : "en-US,en;q=0.9",
            "Cookie" : parsed_cookies}
    
 
    final_result_set = ""
    result_row = ""
    
    for _symbol in _symbol_list:
        
        print("-------------------------------------------------")
        print("Checking: Stock: "+_symbol)
 
        rs = requests.get("https://www1.nseindia.com//marketinfo/sym_map/symbolCount.jsp?symbol="+_symbol,headers=custom_headers)
        _symbolCount = str(rs.text).strip()
        
        #print("After calling symbolCount.jsp: symbolCount="+_symbolCount)
        
        custom_query_params = {
           'symbol': _symbol,
           'segmentLink': _segmentLink,
           'symbolCount': _symbolCount,
           'series': _series,
           'dateRange': _dateRange,
           'fromDate': _fromDate,
           'toDate': _toDate,
           'dataType': _dataType}  
 
        rs = requests.get("https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp", params=custom_query_params,headers=custom_headers)
        parsed_tables = pandas.read_html(rs.text)
        
        #valid_data_flag = True
        #if not ("<span><nobr>Data for" in rs.text): 
        #    valid_data_flag = False 
        #with open("C:\\CG505\\CG505_Programming\\nse-volume-api\\temp\\debug.txt", "a+") as f:
        #    f.write(rs.text+"\n\n\n--------------------")
        

        df = pandas.DataFrame(parsed_tables[0])
        
        
        ## Notoriously last column name is having some weird html new lines causing the whole script to fail
        ## https://stackoverflow.com/questions/56479835/rename-only-the-last-column-in-pandas-dataframe-accounting-for-duplicate-header
        #df = df[df.columns.values.tolist()].replace('\\n','')
        #df.columns = [*df.columns[:-1], 'Percent of Delivery Qty to Traded Qty']
        
        #print("Colunm size: "+str(len(df.columns)) ) 
        
        
        time_stamp_file = _symbol +"_"+str(datetime.datetime.now()).replace(" ","-").replace(":","-").replace(".","-") + ".csv"
        
        df.to_csv("temp/"+time_stamp_file)
        ## Debug info:
        print("Columns names in "+_symbol)
        print(list(df.columns.values))
        
        cols_2 = ['Date', 'Total Traded Quantity', 'DeliverableQty','% Dly Qt toTraded Qty']
        
        

        print("#Columns in stock "+_symbol+" : "+str(len(df.columns)))
        
        if (len(df.columns)!=15): # Work on data if and only the data has 15 cols => Proper data
            #print("#Columns in stock "+_symbol+" is not equal to 15")
            result_row = _symbol+",NSE DATA ERROR,NSE DATA ERROR,NSE DATA ERROR,NSE DATA ERROR\n"
            print("*** Result: "+_symbol+" : "+result_row)
        
        else:
            df2 = df[['Date', 'Total Traded Quantity', 'DeliverableQty','% Dly Qt toTraded Qty']].copy()
            Latest_Date = df2["Date"].to_numpy()[-1]
            result_row = iqrAnomalyCheck(_symbol,Latest_Date,df2)
            print("*** Result: "+_symbol+" : "+result_row)
            
 

        final_result_set = final_result_set + result_row
 
    return final_result_set

In [5]:
def mainFunction():

    fileDir = os.path.dirname(os.path.realpath('__file__'))
    stock_list = []
    with open(os.path.join(fileDir, 'stock-list/stocklist.txt')) as f:
        stock_list = f.read().splitlines()
        
    final_report = fetchDataFromNSE(stock_list, 28)
    
    report_filename = "reports/"+str(datetime.datetime.now()).replace(" ","-").replace(":","-").replace(".","-") + ".csv"
    
    fileDir = os.path.dirname(os.path.realpath('__file__'))
     
    with open(os.path.join(fileDir, report_filename), "w+") as f:
         f.write("STOCK,DATE,TRADING,DELIVERY,DELIVERY PERCENTAGE\n")
         f.write(final_report)
 
    print("\n-------------FINAL REPORT---------------\n")
    print(final_report)
    
    #send_email(os.path.join(fileDir, report_filename))

In [6]:
mainFunction()

date from  04-03-2022 <class 'str'>
date today  01-04-2022 <class 'str'>
-------------------------------------------------
Checking: Stock: AARTIIND
Columns names in AARTIIND
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock AARTIIND : 15
*** Result: AARTIIND : AARTIIND,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: ABBOTINDIA
Columns names in ABBOTINDIA
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock ABBOTINDIA : 15
*** Result: ABBOTINDIA : ABBOTINDIA,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------

Columns names in ZYDUSLIFE
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock ZYDUSLIFE : 15
*** Result: ZYDUSLIFE : ZYDUSLIFE,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: CAMS
Columns names in CAMS
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock CAMS : 15
*** Result: CAMS : CAMS,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: CDSL
Columns names in CDSL
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Las

Columns names in HEROMOTOCO
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock HEROMOTOCO : 15
*** Result: HEROMOTOCO : HEROMOTOCO,01-Apr-2022,ABNOMALLY HIGH TRADING,ABNOMALLY HIGH DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: HINDUNILVR
Columns names in HINDUNILVR
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock HINDUNILVR : 15
*** Result: HINDUNILVR : HINDUNILVR,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: ICICIBANK
Columns names in ICICIBANK
['Symbol', 'Series', 'Date',

Columns names in MINDAIND
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock MINDAIND : 15
*** Result: MINDAIND : MINDAIND,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: MOTHERSUMI
Columns names in MOTHERSUMI
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock MOTHERSUMI : 15
*** Result: MOTHERSUMI : MOTHERSUMI,01-Apr-2022,NORMAL TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: MUTHOOTFIN
Columns names in MUTHOOTFIN
['Symbol', 'Series', 'Date', 'Prev Close', 'Open P

Columns names in TATAMOTORS
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock TATAMOTORS : 15
*** Result: TATAMOTORS : TATAMOTORS,01-Apr-2022,HIGH TRADING,NORMAL DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: TATAPOWER
Columns names in TATAPOWER
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price', 'Last Price', 'Close Price', 'VWAP', 'Total Traded Quantity', 'Turnover', 'No. of Trades', 'DeliverableQty', '% Dly Qt toTraded Qty']
#Columns in stock TATAPOWER : 15
*** Result: TATAPOWER : TATAPOWER,01-Apr-2022,HIGH TRADING,HIGH DELIVERY,NORMAL DELIVERY PERCENTAGE

-------------------------------------------------
Checking: Stock: TCS
Columns names in TCS
['Symbol', 'Series', 'Date', 'Prev Close', 'Open Price', 'High Pric